<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_4_71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import fastai

In [2]:
from fastai.text import * 

In [3]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [4]:
# For veracity
train = read_veracity('modifiedTrain_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [5]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [7]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [8]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [9]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.619275,4.265988,0.288980,00:43


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.059934,3.958266,0.326964,00:51


In [12]:
learn.predict("This is a review about", n_words=10)

'This is a review about # Ferguson Police Chief yesterday .'

In [13]:
learn.save_encoder('ft_enc')

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (38400 items)
x: TextList
xxbos xxunk _ amen 🙏,xxbos @mashable i fl with xxmaj germanwings 2 weeks ago and have done so often connecting to xxup lh flights in xxmaj frankfurt . xxmaj good crews and equipment,xxbos @matthewbarton @stephenfry @englishpen @indexcensorship xxmaj the beheadings , rape of children , woman being treated as objects etc , all teachings,xxbos @henryhaste @airlivenet @laprovence @pollyr_aviation what a stupid comment,xxbos @independent i ca n’t explain what i feel after the killing of xxmaj cabu / 
  xxmaj je ne peux pas xxunk ce que le xxunk de xxmaj cabu me fait xxunk .
y: CategoryList
False,False,False,False,True
Path: .;

Valid: LabelList (9601 items)
x: TextList
xxbos xxunk : xxmaj funny how that works .,xxbos @guardiannews " pour fuel on the fire " .. admitting at least there is a fire !,xxbos @fieryreddragon @unclerush @chereenz and why are the terrorists in xxmaj myanmar xxmaj buddhist , hypocrite and c

In [15]:
data_clas.show_batch()

text,target
xxbos @skynewsbreak @kileysky - please show the cartoons that these brave ppl drew instead of murder footage . \n @skynewsbreak xxunk xxrep 4 0 .0 \n @robertacohen003 https : / / t.co / gr8sbnjghu xxunk \n xxmaj police say 12 now dead in # charliehebdo attack including 2 police officers ; prosecutor says gunmen still at large http : / / t.co / xxunk \n xxmaj,True
"xxbos xxunk xxunk 💖 xxunk 💖 xxunk 💖 \n xxunk \n “ @_juliyana : xxmaj my friend 's mom is in xxmaj korea right now and she met xxmaj yonghwa , shooting for his drama . http : / / t.co / xxunk ”",False
"xxbos xxunk xxmaj this xxmaj nov xxunk will be about "" the man who lies here "" & & "" the man who died here "" . i will be there . xxmaj my boys will be there . # cenotaph",False
xxbos “ @cbcnews : xxup breaking xxup news : xxmaj soldier shot at xxmaj national xxmaj war xxmaj memorial in xxmaj ottawa http : / / t.co / xxunk ” . xxmaj be safe out there folks in xxmaj uniform,True
"xxbos @rt_com xxmaj well done ( xxup not ) for promoting that # racist far - right # xxup xxunk party , it will really help keep us safe , as the death penalty does xxup not do ! !",False


In [16]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.473857,0.438786,0.802416,00:41


In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.418497,0.381430,0.826268,00:45


In [18]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.403631,0.364918,0.834496,01:20
1,0.383236,0.336835,0.846370,01:19
2,0.314842,0.295289,0.864806,01:20
3,0.292799,0.264476,0.876159,01:19
4,0.251424,0.252963,0.886991,01:20
5,0.226462,0.248147,0.893136,01:20
6,0.178244,0.246208,0.890532,01:20
7,0.160025,0.261761,0.893553,01:20
8,0.141560,0.256312,0.895428,01:19
9,0.148654,0.259659,0.897094,01:19


In [19]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
accuracy_score(test['label'], l)

0.7124715615305067

In [22]:
f1_score(test['label'], l, average='macro')

0.4783300758382409

In [23]:
learn.export('veracity.pkl')